In [129]:
import numpy as np
import pandas as pd
import itertools

In [130]:
# Create a 100x100 array at elevation 0
numParticlesGridLength = 10
numAttr = 5
waterMolecules = np.zeros((numParticlesGridLength**3, numAttr), dtype=int)

df = pd.DataFrame(waterMolecules, columns=['x', 'y', 'z', 'temp', 'state'])
print(df)

     x  y  z  temp  state
0    0  0  0     0      0
1    0  0  0     0      0
2    0  0  0     0      0
3    0  0  0     0      0
4    0  0  0     0      0
..  .. .. ..   ...    ...
995  0  0  0     0      0
996  0  0  0     0      0
997  0  0  0     0      0
998  0  0  0     0      0
999  0  0  0     0      0

[1000 rows x 5 columns]


In [131]:
df['state'] = 'liquid'
print(df)

     x  y  z  temp   state
0    0  0  0     0  liquid
1    0  0  0     0  liquid
2    0  0  0     0  liquid
3    0  0  0     0  liquid
4    0  0  0     0  liquid
..  .. .. ..   ...     ...
995  0  0  0     0  liquid
996  0  0  0     0  liquid
997  0  0  0     0  liquid
998  0  0  0     0  liquid
999  0  0  0     0  liquid

[1000 rows x 5 columns]


In [132]:
combinations = list(itertools.product(range(numParticlesGridLength), repeat=3))
print(combinations)
df['x'], df['y'], df['z'] = zip(*combinations)

print(df)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 0, 5), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 0, 9), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 5, 0), (0, 5, 1), (0, 5, 2), (0, 5, 3), (0, 5, 4), (0, 5, 5), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 6, 0), (0, 6, 1), (0, 6, 2), (0, 6, 3), (0, 6, 4), (0, 6, 5), (0, 6, 6), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 7, 0), (0, 7, 1), (0, 7, 2), (0, 7, 3), (0, 7, 4), (0, 7, 5), (0, 7, 6), (0, 7, 7), (0, 7, 8), (0, 7, 9), (0, 8, 0), (0, 8, 1), (0, 8, 2), (0, 8, 3), (0, 8, 4), (0, 8, 5), (0, 8, 6), (0, 8, 7), (0, 8, 8), (0, 8, 9), (0, 9, 0)

In [133]:
def warmUpTemp(df):
    random_increments = np.random.randint(0, 5, size=len(df))
    df['temp'] += random_increments
    return df


In [134]:
for i in range(10):
    df = warmUpTemp(df)

# water evaporates at all temp depending on humidity, this doesn't work
df.loc[df['temp'] >= 100, 'state']= 'gas'
print(df[df['state'] == 'gas'])


Empty DataFrame
Columns: [x, y, z, temp, state]
Index: []


In [135]:
def evaporateWater(df):
    df.loc[df['state'] == 'gas', 'elevation'] += 1
    return df

def windBlowing(df):
    df.loc[df['x'] > 0, 'x'] -= 1
    return df



In [138]:
# TEST - all x>5 with z=1 moved up. All x>5 should then be able to evaporate
df.loc[(df['x'] > 5) & (df['z'] == 1), 'z'] += 1

# print(df[(df['x'] > 5) & (df['z'] == 0)])

'''
This finds all the water molecules at x,y,z=0 that have no corresponding molecule at x,y,z=1 ie right above them.
These molecules have "room" to evaporate above them.
'''

# Extract all water @ z = 0 and set it to z=1
waterHeight = df.loc[(df['z'] == 0), ['x', 'y']]
waterHeight['z'] = 1

# print(waterHeight)

# Merge the waterHeight molcules with all the ones originally at z = 1. Keep the index.
waterHeight = pd.merge(waterHeight, df[df['z'] == 1], on=['x', 'y'], how='left', indicator=True).set_axis(waterHeight.index)

# Find all the molecules in the merge that only existed in waterHeight. These are the molecules at water level that have no molecules at z=1. Set their z back to 0
waterHeight.loc[waterHeight['_merge'] == 'left_only', ['z_x']] = 0
# Extract x,y,z=0 that have no molecule at z+1
mergedXY = waterHeight.loc[waterHeight['_merge'] == 'left_only', ['x', 'y', 'z_x']]
print(mergedXY)
print(mergedXY.index)

# Using the index, update them somehow, placeholder state=gas
df.loc[mergedXY.index, 'state'] = 'gas'

df[df['state'] == 'gas']

     x  y  z_x
600  6  0    0
610  6  1    0
620  6  2    0
630  6  3    0
640  6  4    0
650  6  5    0
660  6  6    0
670  6  7    0
680  6  8    0
690  6  9    0
700  7  0    0
710  7  1    0
720  7  2    0
730  7  3    0
740  7  4    0
750  7  5    0
760  7  6    0
770  7  7    0
780  7  8    0
790  7  9    0
800  8  0    0
810  8  1    0
820  8  2    0
830  8  3    0
840  8  4    0
850  8  5    0
860  8  6    0
870  8  7    0
880  8  8    0
890  8  9    0
900  9  0    0
910  9  1    0
920  9  2    0
930  9  3    0
940  9  4    0
950  9  5    0
960  9  6    0
970  9  7    0
980  9  8    0
990  9  9    0
Index([600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730,
       740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870,
       880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990],
      dtype='int64')


,x,y,z,temp,state
600,6,0,0,28,gas
610,6,1,0,18,gas
620,6,2,0,13,gas
630,6,3,0,20,gas
640,6,4,0,17,gas
650,6,5,0,26,gas
660,6,6,0,23,gas
670,6,7,0,22,gas
680,6,8,0,24,gas
690,6,9,0,24,gas


In [ ]:
#